In [ ]:
# DNN_A with 1025 input and 2050 output
# DNN_C with 2050 input and 2050 output
# Half of the data is used to train DNN_A and the other half is given to trained DNN_A to generate data for DNN_C
# DNN_C has it's own cost function(customized)

In [1]:
#import libraries.
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
# from keras.models import Sequential
# from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler
import math

Using TensorFlow backend.


In [2]:
#import libraries.
# import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
#read necessary files, contarins mixed and clean features.
def data_reading(i):
    temp = open( "TIMIT/Organized/concatenated/Mixed" +str(i) + ".p", "rb")
    ftr = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    temp = open( "TIMIT/Organized/concatenated/clean" +str(i) + ".p", "rb")
    target = pickle.load( temp ).T
    temp.close()
    return ftr, target

In [4]:
def phase_reading(i):
    temp = open( "TIMIT/Organized/concatenated/Mixed_phase" +str(i) + ".p", "rb")
    angle = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    return angle

In [5]:
h_A = [1025,1025,1025]
h_C = [4100, 4100, 4100]

In [6]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=160, win_length=400, window='hann')
    return recon

In [7]:
# class Generator(Sequence):
#     # Class is a dataset wrapper for better training performance
#     def __init__(self,start, stop, batch_size=128):
#         h5f = h5py.File('TIMIT/Organized/concatenated/mixed0.hdf5','r')
#         h5f1 = h5py.File('TIMIT/Organized/concatenated/clean0.hdf5','r')
# #         self.start = start
#         self.x, self.y = h5f['mixed0'][:,start:stop].T, h5f1['clean0'][:,start:stop].T
#         self.batch_size = batch_size
#         self.indices = np.arange(self.x.shape[0])

#     def __len__(self):
#         return math.ceil(self.x.shape[0] / self.batch_size)

#     def __getitem__(self, idx):
#         inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
#         batch_x = self.x[inds]
# #         print(batch_x.shape)
#         batch_y = self.y[inds]
#         return batch_x, batch_y
    
#     def on_epoch_end(self):
#         np.random.shuffle(self.indices)

In [ ]:
def da(load_size, data_len):
    global I
    h5f1 = h5py.File('TIMIT/Organized/concatenated/mixed0.hdf5','r')
    h5f2 = h5py.File('TIMIT/Organized/concatenated/clean0.hdf5','r')
    indx = np.arange(0, data_len, load_size)
    indx = shuffle(indx, random_state=0)
    print(indx)
    while True:             #this line is just because keras needs infinite generators
        for I in indx: 
            if data_len-I<load_size:
                Data1 = h5f1['mixed0'][:,I:].T
                Data2 = h5f2['clean0'][:,I:].T
            else:
                Data1 = h5f1['mixed0'][:,I:I+load_size].T
                Data2 = h5f2['clean0'][:,I:I+load_size].T
            yield(Data1, Data2)


In [8]:
# X, y = data_reading(0)
X_dim= 1025
y_dim = 2050
estimator_A = Sequential()
estimator_A.add(Dense(h_A[0], input_dim = X_dim, kernel_initializer='normal', activation='relu'))
estimator_A.add(Dense(h_A[1], kernel_initializer='normal', activation='relu'))
estimator_A.add(Dense(h_A[2], kernel_initializer='normal', activation='relu'))
estimator_A.add(Dense(y_dim, kernel_initializer='normal', activation='relu'))
# Compile model
estimator_A.compile(loss='mean_squared_error', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [9]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed0.hdf5','r')
ftr_shape = h5f['mixed0'].shape
h5f.close()
batch_size = 128
steps_per_epoch = int(ftr_shape[1]/(2*batch_size))

In [10]:
h5f = h5py.File('TIMIT/Organized/concatenated/clean0.hdf5','r')
ftr = h5f['clean0'][:].shape
h5f.close()

In [11]:
ftr_shape

(1025, 939485)

In [10]:
# training_generator = Generator(0, int(ftr_shape[1]/2))

In [11]:
steps_per_epoch

3669

In [12]:
# gg = data_gen(batch_size, int(ftr_shape[0]/2))

In [48]:
# print(next(gg)[0].shape)

(128, 1025)


In [10]:
tic = time.time()
estimator_A.fit_generator(generator=da(batch_size, ftr_shape[1]),
                          steps_per_epoch=steps_per_epoch,
                          epochs=50,
                          verbose=1)
toc  =time.time()


Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 285/3670 [=>............................] - ETA: 4:05 - loss: 0.1895

KeyboardInterrupt: 

In [29]:
print('time_of_train =', (toc - tic)/60)

time_of_train = 79.79761300086975


In [30]:
estimator_A.save('Models/Two_stage/trained_model.h5')

In [31]:
estimator_A.save_weights('Models/Two_stage/weights.h5')

In [7]:
estimator_A = load_model('Models/Two_stage/trained_model4_128.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [78]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [105]:
test_point_start0 = 2000000
test_point_stop0 = 2001000

test_point_start = 469750
test_point_stop = 469850


In [106]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed3_sec.hdf5','r')
test_input = h5f['mixed3_sec'][:,test_point_start0 : test_point_stop0]
h5f.close()

In [107]:
# test_input

In [108]:
h5f = h5py.File('TIMIT/Organized/concatenated/clean3_sec.hdf5','r')
test_target = h5f['clean3_sec'][:, test_point_start0 : test_point_stop0]
h5f.close()

In [109]:
h5f = h5py.File('TIMIT/Organized/concatenated/phase3_sec.hdf5','r')
angle = h5f['phase3_sec'][:,test_point_start0 : test_point_stop0]
h5f.close()

In [110]:
print(test_input.shape)
print(test_target.shape)
print(angle.shape)

(257, 1000)
(514, 1000)
(257, 1000)


In [111]:
prediction = estimator_A.predict(test_input.T)

In [112]:
prediction.shape

(1000, 514)

In [113]:
fft_samples = 257

In [114]:
# prediction[0,fft_samples:fft_samples*2]

In [115]:
# prediction[0,0:fft_samples]

In [116]:
s1 = reconstruct(prediction[:,0:fft_samples], angle.T)

In [117]:
clean_recon = reconstruct(test_target[0:fft_samples,:].T, angle.T)

In [118]:
mixed_recon = reconstruct(test_input[0:fft_samples,:].T, angle.T)

In [119]:
sd.play(s1*10, 16000)

In [130]:
sd.play(clean_recon*10, 16000)

In [121]:
sd.play(mixed_recon*10, 16000)

In [122]:
st_recon = stoi(s1, clean_recon, 16000, extended=False)

In [123]:
st_recon

0.6907444744607958

In [124]:
st_mixed = stoi(mixed_recon, clean_recon, 16000, extended=False)

In [125]:
st_mixed

0.747816166328542

In [126]:
sdr_recon, sir_recon, sar_recon, perm_recon = separation.bss_eval_sources(s1, clean_recon, compute_permutation=True)

In [127]:
sdr_mixed, sir_mixed, sar_mixed, perm_mixed = separation.bss_eval_sources(mixed_recon, clean_recon, compute_permutation=True)

In [128]:
print('SDR_recon=', sdr_recon)
print('SDR_mixed=', sdr_mixed)

SDR_recon= [5.62683457]
SDR_mixed= [7.57235015]


In [ ]:
1025:2050